In [1]:
import os
import torch

In [2]:
class ClassificationTester:
    def __init__(self, project_name, model, test_data_loader, transforms, checkpoint_file_path):
        self.project_name = project_name
        self.model = model
        self.test_data_loader = test_data_loader
        self.transforms = transforms
    
        self.latest_file_path = os.path.join(
            checkpoint_file_path, f"{project_name}_checkpoint_latest.pt"
        )
        
        print("MODEL FILE: {0}".format(self.latest_file_path))
        
        self.model.load_state_dict(torch.load(self.latest_file_path, map_location=torch.device('cpu')))
    
    def test(self):
        self.model.eval()
        
        num_corrects_test = 0
        num_tested_samples = 0
    
        with torch.no_grad():
            for test_batch in self.test_data_loader:
                input_test, target_test = test_batch
        
                input_test = self.transforms(input_test)
        
                output_test = self.model(input_test)
        
                predicted_test = torch.argmax(output_test, dim=1)
                num_corrects_test += torch.sum(torch.eq(predicted_test, target_test))
        
                num_tested_samples += len(input_test)
        
            test_accuracy = 100.0 * num_corrects_test / num_tested_samples
        
        print(f"TEST RESULTS: {test_accuracy:6.3f}%")
    
    def test_single(self, input_test):
        self.model.eval()
        
        with torch.no_grad():
            input_test = self.transforms(input_test)
            
            output_test = self.model(input_test)
            predicted_test = torch.argmax(output_test, dim=1)
        
        return predicted_test.item()
